In [3]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import os

print(f"Working directory: {os.getcwd()}")

# Setup the Open-Meteo API client
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"

# --- 1. DEFINE MULTIPLE LOCATIONS HERE ---
params = {
    "latitude": [40.0, 34.05, 51.50],  # Example: Latitudes for 3 locations
    "longitude": [-85.0, -118.25, -0.12], # Corresponding longitudes
    "start_date": "1970-01-01",
    "end_date": "2025-11-09",
    "daily": ["temperature_2m_mean", "precipitation_sum", "dew_point_2m_mean", "cloud_cover_mean", "cloud_cover_max", "dew_point_2m_max", "dew_point_2m_min", "cloud_cover_min", "apparent_temperature_mean", "et0_fao_evapotranspiration_sum", "relative_humidity_2m_mean", "relative_humidity_2m_min", "relative_humidity_2m_max", "snowfall_water_equivalent_sum", "pressure_msl_mean", "pressure_msl_max", "pressure_msl_min", "surface_pressure_mean", "surface_pressure_min", "winddirection_10m_dominant", "surface_pressure_max", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_speed_10m_mean", "wind_gusts_10m_mean", "wind_gusts_10m_min", "wind_speed_10m_min", "wet_bulb_temperature_2m_mean", "wet_bulb_temperature_2m_max", "vapour_pressure_deficit_max", "soil_moisture_0_to_7cm_mean", "soil_moisture_0_to_100cm_mean", "wet_bulb_temperature_2m_min", "soil_moisture_7_to_28cm_mean", "soil_moisture_28_to_100cm_mean", "soil_temperature_0_to_100cm_mean", "soil_temperature_0_to_7cm_mean", "soil_temperature_28_to_100cm_mean", "soil_temperature_7_to_28cm_mean"],
    "timezone": "auto",
}
responses = openmeteo.weather_api(url, params=params)

# List to hold dataframes for all locations before combining
all_locations_data = []

# --- 2. LOOP THROUGH ALL LOCATIONS ---
for i, response in enumerate(responses):
    print(f"Processing location {i+1}: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
    
    daily = response.Daily()
    
    # (Your existing variable extraction code remains mostly the same)
    daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
    daily_dew_point_2m_mean = daily.Variables(2).ValuesAsNumpy()
    daily_cloud_cover_mean = daily.Variables(3).ValuesAsNumpy()
    daily_cloud_cover_max = daily.Variables(4).ValuesAsNumpy()
    daily_dew_point_2m_max = daily.Variables(5).ValuesAsNumpy()
    daily_dew_point_2m_min = daily.Variables(6).ValuesAsNumpy()
    daily_cloud_cover_min = daily.Variables(7).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(8).ValuesAsNumpy()
    daily_et0_fao_evapotranspiration_sum = daily.Variables(9).ValuesAsNumpy()
    daily_relative_humidity_2m_mean = daily.Variables(10).ValuesAsNumpy()
    daily_relative_humidity_2m_min = daily.Variables(11).ValuesAsNumpy()
    daily_relative_humidity_2m_max = daily.Variables(12).ValuesAsNumpy()
    daily_snowfall_water_equivalent_sum = daily.Variables(13).ValuesAsNumpy()
    daily_pressure_msl_mean = daily.Variables(14).ValuesAsNumpy()
    daily_pressure_msl_max = daily.Variables(15).ValuesAsNumpy()
    daily_pressure_msl_min = daily.Variables(16).ValuesAsNumpy()
    daily_surface_pressure_mean = daily.Variables(17).ValuesAsNumpy()
    daily_surface_pressure_min = daily.Variables(18).ValuesAsNumpy()
    daily_winddirection_10m_dominant = daily.Variables(19).ValuesAsNumpy()
    daily_surface_pressure_max = daily.Variables(20).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(21).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(22).ValuesAsNumpy()
    daily_wind_speed_10m_mean = daily.Variables(23).ValuesAsNumpy()
    daily_wind_gusts_10m_mean = daily.Variables(24).ValuesAsNumpy()
    daily_wind_gusts_10m_min = daily.Variables(25).ValuesAsNumpy()
    daily_wind_speed_10m_min = daily.Variables(26).ValuesAsNumpy()
    daily_wet_bulb_temperature_2m_mean = daily.Variables(27).ValuesAsNumpy()
    daily_wet_bulb_temperature_2m_max = daily.Variables(28).ValuesAsNumpy()
    daily_vapour_pressure_deficit_max = daily.Variables(29).ValuesAsNumpy()
    daily_soil_moisture_0_to_7cm_mean = daily.Variables(30).ValuesAsNumpy()
    daily_soil_moisture_0_to_100cm_mean = daily.Variables(31).ValuesAsNumpy()
    daily_wet_bulb_temperature_2m_min = daily.Variables(32).ValuesAsNumpy()
    daily_soil_moisture_7_to_28cm_mean = daily.Variables(33).ValuesAsNumpy()
    daily_soil_moisture_28_to_100cm_mean = daily.Variables(34).ValuesAsNumpy()
    daily_soil_temperature_0_to_100cm_mean = daily.Variables(35).ValuesAsNumpy()
    daily_soil_temperature_0_to_7cm_mean = daily.Variables(36).ValuesAsNumpy()
    daily_soil_temperature_28_to_100cm_mean = daily.Variables(37).ValuesAsNumpy()
    daily_soil_temperature_7_to_28cm_mean = daily.Variables(38).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    # Assign all variables to the dictionary
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["dew_point_2m_mean"] = daily_dew_point_2m_mean
    daily_data["cloud_cover_mean"] = daily_cloud_cover_mean
    daily_data["cloud_cover_max"] = daily_cloud_cover_max
    daily_data["dew_point_2m_max"] = daily_dew_point_2m_max
    daily_data["dew_point_2m_min"] = daily_dew_point_2m_min
    daily_data["cloud_cover_min"] = daily_cloud_cover_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["et0_fao_evapotranspiration_sum"] = daily_et0_fao_evapotranspiration_sum
    daily_data["relative_humidity_2m_mean"] = daily_relative_humidity_2m_mean
    daily_data["relative_humidity_2m_min"] = daily_relative_humidity_2m_min
    daily_data["relative_humidity_2m_max"] = daily_relative_humidity_2m_max
    daily_data["snowfall_water_equivalent_sum"] = daily_snowfall_water_equivalent_sum
    daily_data["pressure_msl_mean"] = daily_pressure_msl_mean
    daily_data["pressure_msl_max"] = daily_pressure_msl_max
    daily_data["pressure_msl_min"] = daily_pressure_msl_min
    daily_data["surface_pressure_mean"] = daily_surface_pressure_mean
    daily_data["surface_pressure_min"] = daily_surface_pressure_min
    daily_data["winddirection_10m_dominant"] = daily_winddirection_10m_dominant
    daily_data["surface_pressure_max"] = daily_surface_pressure_max
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_speed_10m_mean"] = daily_wind_speed_10m_mean
    daily_data["wind_gusts_10m_mean"] = daily_wind_gusts_10m_mean
    daily_data["wind_gusts_10m_min"] = daily_wind_gusts_10m_min
    daily_data["wind_speed_10m_min"] = daily_wind_speed_10m_min
    daily_data["wet_bulb_temperature_2m_mean"] = daily_wet_bulb_temperature_2m_mean
    daily_data["wet_bulb_temperature_2m_max"] = daily_wet_bulb_temperature_2m_max
    daily_data["vapour_pressure_deficit_max"] = daily_vapour_pressure_deficit_max
    daily_data["soil_moisture_0_to_7cm_mean"] = daily_soil_moisture_0_to_7cm_mean
    daily_data["soil_moisture_0_to_100cm_mean"] = daily_soil_moisture_0_to_100cm_mean
    daily_data["wet_bulb_temperature_2m_min"] = daily_wet_bulb_temperature_2m_min
    daily_data["soil_moisture_7_to_28cm_mean"] = daily_soil_moisture_7_to_28cm_mean
    daily_data["soil_moisture_28_to_100cm_mean"] = daily_soil_moisture_28_to_100cm_mean
    daily_data["soil_temperature_0_to_100cm_mean"] = daily_soil_temperature_0_to_100cm_mean
    daily_data["soil_temperature_0_to_7cm_mean"] = daily_soil_temperature_0_to_7cm_mean
    daily_data["soil_temperature_28_to_100cm_mean"] = daily_soil_temperature_28_to_100cm_mean
    daily_data["soil_temperature_7_to_28cm_mean"] = daily_soil_temperature_7_to_28cm_mean

    # Create DataFrame for THIS location
    location_df = pd.DataFrame(data = daily_data)
    
    # --- 3. ADD IDENTIFIERS ---
    # Add lat/lon columns so we know which location this data belongs to in the final file
    location_df["Latitude"] = response.Latitude()
    location_df["Longitude"] = response.Longitude()
    location_df["Location_ID"] = i + 1  # Optional: a simple 1, 2, 3 ID
    
    # Add to our master list
    all_locations_data.append(location_df)

# --- 4. COMBINE AND SAVE ---
# Concatenate all individual location dataframes into one big one
final_dataframe = pd.concat(all_locations_data, ignore_index=True)

filename = "combined_daily_weather_data.csv"
final_dataframe.to_csv(filename, index=False)
print(f"\nSaved combined data for {len(responses)} locations to: {filename}")

Working directory: c:\Users\roosd\Downloads\econometrie jaar 3\TIC\TIC-Quantitative-Research-Group-1\data_comodities
Processing location 1: 39.96°N -84.97°E
Processing location 2: 34.06°N -118.24°E
Processing location 3: 51.49°N -0.16°E

Saved combined data for 3 locations to: combined_daily_weather_data.csv
